# XLSX creation FPI / Image Overall

### General Settings

In [102]:
import pandas as pd
import numpy as np
#import re as re
#import itertools as it
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from pandas.io import sql
#from shutil import copyfile
#from subprocess import Popen
import winsound

In [103]:
#pd.set_option('display.max_columns', 1000)
#pd.set_option('display.max_rows', 10)

### Prerequisites

In [104]:
p_year=2016
p_countrycode='THA' #not used at the moment, as we loop trough all countries for now

#### Set directories

In [105]:
p_dir=r'\\TSHAMFIL901\iClick\Projects\Allianz\DB\Chartpack\XLSX'.replace(chr(92),'/')

In [106]:
p_subdir=p_dir +'/'+ r'data'.replace(chr(92),'/')
p_xlsx = 'MMX'  #+str(p_year)[-2:]

#### Prepare DB connection

In [107]:
engine_dv_bpm = create_engine('mssql+pyodbc://TSMMHSQVS901/DV_BPM?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes')

In [108]:
#Hilfstabelle zum Bilden von Vorwellenvergleichen (Vorwelle statt Vorjahr -> Wenn keine Vorjahresbefragung, dann Vorvorjahr)
#Die ganze Logik steckt im View "WaveID_pre" in der Datenbank
df_prewave=pd.read_sql_query(
    """SELECT [CountryID]
      ,[LoBID]
      ,wp.[WaveID]
      ,[WaveID_pre]
  FROM [DV_BPM].[dbo].[WaveID_pre] wp
  JOIN [DV_BPM].[dbo].[Wave] w
  ON wp.WaveID=w.WaveID
  WHERE w.Year = 
    """ + str(p_year), engine_dv_bpm)

In [109]:
df_prewave_cross=df_prewave.groupby(['CountryID','WaveID'],as_index=False)['WaveID_pre'].max().copy()
#df_prewave_cross.columns=['CountryID','WaveID','PreWaveID']
#df_prewave_cross,df_prewave_cross.columns
df_prewave_cross['WaveID_pre']=df_prewave_cross['WaveID_pre'].fillna(0).astype(int)
#df_prewave_cross

### Data preparation

#### FPI

In [110]:
df=pd.read_sql_query(
    """SELECT *
    FROM DV_BPM.dbo.xls_FPI WHERE CC is not null
    """, engine_dv_bpm)

In [111]:
#Marktdurchschnitt für SQR4 rechnen:
dfg=df.groupby(['CountryID','LoBID','WaveID'],as_index=False)['SQR4'].mean()
#... und Ergebnis in df schreiben:
df=pd.merge(left=df, right=dfg, left_on=('CountryID','LoBID','WaveID'), right_on=('CountryID','LoBID','WaveID'), how='left', suffixes=['','_mean'])

In [112]:
#FPI rechnen (Verhältnis SQR4 zum Marktddurchschnitt in Prozent):
df['FPI']=(df.SQR4/df.SQR4_mean)*100
#Werte für Vorwellenvergleiche erzeugen:
df_delta=df[['CountryID','LoBID','BrandID','WaveID','FPI','Q7_all_AwUnaided','Q8_AwAided']].copy()
#Vorwelle ermitteln und in df eintragen:
df=pd.merge(left=df, right=df_prewave, left_on=('CountryID','LoBID','WaveID'), right_on=('CountryID','LoBID','WaveID'), how='left', suffixes=['','_XXX'])
#Vorwellenwerte in df eintragen:
df=pd.merge(left=df, right=df_delta, left_on=('CountryID','LoBID','BrandID','WaveID_pre'), right_on=('CountryID','LoBID','BrandID','WaveID'), how='left', suffixes=['','_PW'])
#Veränderungen zur Vorwelle ('Previous Wave'->_PW) ermitteln und als Felder in df eintragen (Suffix: _delta):
df['FPI_delta']=df.FPI-df.FPI_PW
df['Q7_all_AwUnaided_delta']=df.Q7_all_AwUnaided-df.Q7_all_AwUnaided_PW
df['Q8_AwAided_delta']=df.Q8_AwAided-df.Q8_AwAided_PW

In [113]:
df.drop(['WaveID_PW','WaveID_pre','CountryID','WaveID'], axis=1, inplace=True)

In [114]:
dfp=df[(df.Year>p_year-3)&(df.Year<=p_year)].pivot_table(index=['CC','Country','LoBID','BrandID','Brand'], columns='Year',values='FPI', aggfunc='first').copy()
dfp.reset_index(inplace=True)

In [115]:
df_fpi=df.copy()
df_fpi_yrs=dfp.copy()

#### Image Overall

In [116]:
df=pd.read_sql_query(
    """SELECT *
    FROM DV_BPM.dbo.XLS_ImagePerceptionOverall
    """, engine_dv_bpm)

In [117]:
dfp=df.pivot_table(index=['CountryID','CC','Country','WaveID','Year','BrandID','Brand'],columns='Customer',values=['Market','BrandValue'])
dfp.reset_index(inplace=True)
df_im_ov=dfp.copy()
#Umwandeln des MultiIndex in den Columns in einen normalen Index:
#https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns
df_im_ov.columns = ['_'.join(col).strip() for col in df_im_ov.columns.values]
#Entfernen der gerade erzeugten Unterstriche für die Columns, die kein zweite Level hatten (also Unterstrich am Ende):
df_im_ov.columns=df_im_ov.columns.str.replace('_$','')
df_im_ov.columns=df_im_ov.columns.str.replace('-','_')

In [118]:
df_im_ov

CountryID       CC             Country  WaveID  Year  BrandID  \
0            1      POL              Poland      16  2016     5001   
1            1      POL              Poland      16  2016     5053   
2            1      POL              Poland      16  2016     5105   
3            1      POL              Poland      16  2016     5155   
4            1      POL              Poland      16  2016     5211   
5            1      POL              Poland      16  2016     5272   
6            1      POL              Poland      16  2016     5634   
7            1      POL              Poland      16  2016     5663   
8            1      POL              Poland      16  2016     5664   
9            1      POL              Poland      16  2016     5665   
10           2      CZE      Czech Republic      16  2016     5009   
11           2      CZE      Czech Republic      16  2016     5053   
12           2      CZE      Czech Republic      16  2016     5070   
13           2      CZE      Czech Republic      16  2016     5071   
14           2      CZE      Czech Republic      16  2016     5073   
15           2      CZE      Czech Republic      16  2016     5096   
16           2      CZE      Czech Republic      16  2016     5105   
17           2      CZE      Czech Republic      16  2016     5130   
18           2      CZE      Czech Republic      16  2016     5144   
19           2      CZE      Czech Republic      16  2016     5262   
20           4      ROM             Romania      16  2016     5013   
21           4      ROM             Romania      16  2016     5047   
22           4      ROM             Romania      16  2016     5105   
23           4      ROM             Romania      16  2016     5109   
24           4      ROM             Romania      16  2016     5113   
25           4      ROM             Romania      16  2016     5193   
26           4      ROM             Romania      16  2016     5262   
27           4      ROM             Romania      16  2016     5578   
28           4      ROM             Romania      16  2016     5675   
29           4      ROM             Romania      16  2016     5676   
..         ...      ...                 ...     ...   ...      ...   
240         93  AUS_BRB  Australia Brisbane      16  2016     5001   
241         93  AUS_BRB  Australia Brisbane      16  2016     5019   
242         93  AUS_BRB  Australia Brisbane      16  2016     5107   
243         93  AUS_BRB  Australia Brisbane      16  2016     5217   
244         93  AUS_BRB  Australia Brisbane      16  2016     5479   
245         93  AUS_BRB  Australia Brisbane      16  2016     5555   
246         93  AUS_BRB  Australia Brisbane      16  2016     5558   
247         93  AUS_BRB  Australia Brisbane      16  2016     5559   
248         93  AUS_BRB  Australia Brisbane      16  2016     5560   
249         93  AUS_BRB  Australia Brisbane      16  2016     5708   
250         94  AUS_PER     Australia Perth      16  2016     5001   
251         94  AUS_PER     Australia Perth      16  2016     5019   
252         94  AUS_PER     Australia Perth      16  2016     5107   
253         94  AUS_PER     Australia Perth      16  2016     5216   
254         94  AUS_PER     Australia Perth      16  2016     5479   
255         94  AUS_PER     Australia Perth      16  2016     5555   
256         94  AUS_PER     Australia Perth      16  2016     5558   
257         94  AUS_PER     Australia Perth      16  2016     5559   
258         94  AUS_PER     Australia Perth      16  2016     5560   
259         94  AUS_PER     Australia Perth      16  2016     5708   
260         95  AUS_ADE  Australia Adelaide      16  2016     5001   
261         95  AUS_ADE  Australia Adelaide      16  2016     5019   
262         95  AUS_ADE  Australia Adelaide      16  2016     5107   
263         95  AUS_ADE  Australia Adelaide      16  2016     5215   
264         95  AUS_ADE  Australia Adelaide      16  2016     5479   
265         95 

In [119]:
#Werte für Vorwellenvergleiche erzeugen:
df_im_ov_delta=df_im_ov[['CountryID','WaveID','BrandID','BrandValue_Customers','BrandValue_Non_Customers']].copy()
#Vorwelle ermitteln und in df eintragen:
df_im_ov=pd.merge(left=df_im_ov, right=df_prewave_cross, left_on=('CountryID','WaveID'), right_on=('CountryID','WaveID'), how='left', suffixes=['','_XXX'])
#Vorwellenwerte in df eintragen:
df_im_ov=pd.merge(left=df_im_ov, right=df_im_ov_delta, left_on=('CountryID','BrandID','WaveID_pre'), right_on=('CountryID','BrandID','WaveID'), how='left', suffixes=['','_PW'])
#Veränderungen zur Vorwelle ('Previous Wave'->_PW) ermitteln und als Felder in df eintragen (Suffix: _delta):
df_im_ov['BrandValue_Customers_PW_delta']=df_im_ov.BrandValue_Customers-df_im_ov.BrandValue_Customers_PW
df_im_ov['BrandValue_Non_Customers_delta']=df_im_ov.BrandValue_Non_Customers-df_im_ov.BrandValue_Non_Customers_PW
df_im_ov

CountryID       CC             Country  WaveID  Year  BrandID  \
0            1      POL              Poland      16  2016     5001   
1            1      POL              Poland      16  2016     5053   
2            1      POL              Poland      16  2016     5105   
3            1      POL              Poland      16  2016     5155   
4            1      POL              Poland      16  2016     5211   
5            1      POL              Poland      16  2016     5272   
6            1      POL              Poland      16  2016     5634   
7            1      POL              Poland      16  2016     5663   
8            1      POL              Poland      16  2016     5664   
9            1      POL              Poland      16  2016     5665   
10           2      CZE      Czech Republic      16  2016     5009   
11           2      CZE      Czech Republic      16  2016     5053   
12           2      CZE      Czech Republic      16  2016     5070   
13           2      CZE      Czech Republic      16  2016     5071   
14           2      CZE      Czech Republic      16  2016     5073   
15           2      CZE      Czech Republic      16  2016     5096   
16           2      CZE      Czech Republic      16  2016     5105   
17           2      CZE      Czech Republic      16  2016     5130   
18           2      CZE      Czech Republic      16  2016     5144   
19           2      CZE      Czech Republic      16  2016     5262   
20           4      ROM             Romania      16  2016     5013   
21           4      ROM             Romania      16  2016     5047   
22           4      ROM             Romania      16  2016     5105   
23           4      ROM             Romania      16  2016     5109   
24           4      ROM             Romania      16  2016     5113   
25           4      ROM             Romania      16  2016     5193   
26           4      ROM             Romania      16  2016     5262   
27           4      ROM             Romania      16  2016     5578   
28           4      ROM             Romania      16  2016     5675   
29           4      ROM             Romania      16  2016     5676   
..         ...      ...                 ...     ...   ...      ...   
240         93  AUS_BRB  Australia Brisbane      16  2016     5001   
241         93  AUS_BRB  Australia Brisbane      16  2016     5019   
242         93  AUS_BRB  Australia Brisbane      16  2016     5107   
243         93  AUS_BRB  Australia Brisbane      16  2016     5217   
244         93  AUS_BRB  Australia Brisbane      16  2016     5479   
245         93  AUS_BRB  Australia Brisbane      16  2016     5555   
246         93  AUS_BRB  Australia Brisbane      16  2016     5558   
247         93  AUS_BRB  Australia Brisbane      16  2016     5559   
248         93  AUS_BRB  Australia Brisbane      16  2016     5560   
249         93  AUS_BRB  Australia Brisbane      16  2016     5708   
250         94  AUS_PER     Australia Perth      16  2016     5001   
251         94  AUS_PER     Australia Perth      16  2016     5019   
252         94  AUS_PER     Australia Perth      16  2016     5107   
253         94  AUS_PER     Australia Perth      16  2016     5216   
254         94  AUS_PER     Australia Perth      16  2016     5479   
255         94  AUS_PER     Australia Perth      16  2016     5555   
256         94  AUS_PER     Australia Perth      16  2016     5558   
257         94  AUS_PER     Australia Perth      16  2016     5559   
258         94  AUS_PER     Australia Perth      16  2016     5560   
259         94  AUS_PER     Australia Perth      16  2016     5708   
260         95  AUS_ADE  Australia Adelaide      16  2016     5001   
261         95  AUS_ADE  Australia Adelaide      16  2016     5019   
262         95  AUS_ADE  Australia Adelaide      16  2016     5107   
263         95  AUS_ADE  Australia Adelaide      16  2016     5215   
264         95  AUS_ADE  Australia Adelaide      16  2016     5479   
265         95 

#### Touchpoint Details

In [120]:
df=pd.read_sql_query(
    """SELECT *
    FROM DV_BPM.dbo.XLS_TouchpointGroups
    """, engine_dv_bpm)

In [121]:
dfp=df.pivot_table(index=['CC','Country','Year','BrandID','Brand','Allianz'],columns='TP_group',values=['tp'])
dfp.reset_index(inplace=True)
df_tp=dfp.copy()
#Umwandeln des MultiIndex in den Columns in einen normalen Index:
#https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns
df_tp.columns = ['_'.join(col).strip() for col in df_tp.columns.values]
#Entfernen der gerade erzeugten Unterstriche für die Columns, die kein zweite Level hatten (also Unterstrich am Ende):
df_tp.columns=df_tp.columns.str.replace('_$','')
df_tp.columns=df_tp.columns.str.replace('tp_','')
df_tp=df_tp[['CC','Country','Year','BrandID','Brand','Allianz','Paid','Owned','Earned']].copy()
df_tpa=df_tp[df_tp.Allianz=='Allianz'][['CC','Country','Year','Allianz','Brand','Paid','Owned','Earned']]
df_tpa.rename(columns={'Allianz': 'KPItype'}, inplace=True)
df_tpa['KPIsort']=1
#df_tpa['Total']=df_tpa.Earned+df_tpa.Owned+df_tpa.Paid
df_tp['Total']=df_tp.Earned+df_tp.Owned+df_tp.Paid
df_tp['Paid%']=df_tp.Paid/df_tp.Total
df_tp['Owned%']=df_tp.Owned/df_tp.Total
df_tp['Earned%']=df_tp.Earned/df_tp.Total
df_tpg=df_tp.groupby(['CC','Country','Year'],as_index=False)['Paid','Owned','Earned'].mean()
df_tpg['KPItype']='Market'
df_tpg['KPIsort']=2
df_tpg['Brand']=''
df_tpm=df_tp.groupby(['CC','Country','Year'],as_index=False)['Total'].max()
df_tpm=pd.merge(left=df_tpm, right=df_tp, left_on=('CC','Country','Year','Total'), right_on=('CC','Country','Year','Total'), how='left', suffixes=['','_X'])
df_tpm['KPItype']='Highest Score'
df_tpm['KPIsort']=3
df_kpi=df_tpm[['CC','Country','Year','KPItype','KPIsort','Brand','Paid','Owned','Earned','Total']].copy()
df_kpi=pd.concat([df_kpi,df_tpa,df_tpg],ignore_index=True)
df_kpi['Total']=df_kpi.Earned+df_kpi.Owned+df_kpi.Paid
df_kpi['Paid%']=df_kpi.Paid/df_kpi.Total
df_kpi['Owned%']=df_kpi.Owned/df_kpi.Total
df_kpi['Earned%']=df_kpi.Earned/df_kpi.Total
df_tp_kpi=df_kpi[['CC','Country','Year','KPItype','KPIsort','Brand','Paid','Owned','Earned','Total','Paid%','Owned%','Earned%']]

#### PIM vs Customer share

In [122]:
df_pim=pd.read_sql_query("""
    SELECT 
       [CC]
      ,[CountryID] 
      ,[Country]
      ,[WaveID]
      ,[Year]
      ,[BrandID]
      ,[Brand]
      ,[PIM]
      ,[CMS]
      ,[Delta]
      ,[DeltaPositiv]
      ,[Rank] FROM [DV_BPM].[dbo].[XLS_Brand_PIMvsCS]""", engine_dv_bpm)
#df_pim

In [123]:
#Werte für Vorwellenvergleiche erzeugen:
df_pim_delta=df_pim[['CountryID','WaveID','BrandID','PIM','Rank']].copy()
#Vorwelle ermitteln und in df eintragen:
df_pim=pd.merge(left=df_pim, right=df_prewave_cross, left_on=('CountryID','WaveID'), right_on=('CountryID','WaveID'), how='left', suffixes=['','_XXX'])
#Vorwellenwerte in df eintragen:
df_pim=pd.merge(left=df_pim, right=df_pim_delta, left_on=('CountryID','BrandID','WaveID_pre'), right_on=('CountryID','BrandID','WaveID'), how='left', suffixes=['','_PW'])
#Veränderungen zur Vorwelle ('Previous Wave'->_PW) ermitteln und als Felder in df eintragen (Suffix: _delta):
df_pim['PIM_PW_delta']=df_pim.PIM-df_pim.PIM_PW
df_pim['Rank_PW_delta']=df_pim.Rank_PW-df_pim.Rank #!!! Rangvergleich ist umgekehrt zu "normalen" Größen
df_pim


CC  CountryID             Country  WaveID  Year  BrandID  \
0        POL          1              Poland      16  2016     5211   
1        POL          1              Poland      16  2016     5272   
2        POL          1              Poland      16  2016     5001   
3        POL          1              Poland      16  2016     5053   
4        POL          1              Poland      16  2016     5664   
5        POL          1              Poland      16  2016     5634   
6        POL          1              Poland      16  2016     5665   
7        POL          1              Poland      16  2016     5105   
8        POL          1              Poland      16  2016     5155   
9        POL          1              Poland      16  2016     5663   
10       POL          1              Poland      16  2016     5079   
11       POL          1              Poland      16  2016     5635   
12       POL          1              Poland      16  2016     5581   
13       POL          1              Poland      16  2016     5207   
14       POL          1              Poland      16  2016     5353   
15       CZE          2      Czech Republic      16  2016     5071   
16       CZE          2      Czech Republic      16  2016     5144   
17       CZE          2      Czech Republic      16  2016     5009   
18       CZE          2      Czech Republic      16  2016     5096   
19       CZE          2      Czech Republic      16  2016     5105   
20       CZE          2      Czech Republic      16  2016     5053   
21       CZE          2      Czech Republic      16  2016     5073   
22       CZE          2      Czech Republic      16  2016     5070   
23       CZE          2      Czech Republic      16  2016     5130   
24       CZE          2      Czech Republic      16  2016     5262   
25       CZE          2      Czech Republic      16  2016     5141   
26       CZE          2      Czech Republic      16  2016     5683   
27       CZE          2      Czech Republic      16  2016     5640   
28       CZE          2      Czech Republic      16  2016     5500   
29       CZE          2      Czech Republic      16  2016     5682   
..       ...        ...                 ...     ...   ...      ...   
435  AUS_PER         94     Australia Perth      16  2016     5216   
436  AUS_PER         94     Australia Perth      16  2016     5001   
437  AUS_PER         94     Australia Perth      16  2016     5019   
438  AUS_PER         94     Australia Perth      16  2016     5213   
439  AUS_PER         94     Australia Perth      16  2016     5479   
440  AUS_PER         94     Australia Perth      16  2016     5709   
441  AUS_PER         94     Australia Perth      16  2016     5708   
442  AUS_PER         94     Australia Perth      16  2016     5107   
443  AUS_PER         94     Australia Perth      16  2016     5555   
444  AUS_PER         94     Australia Perth      16  2016     5478   
445  AUS_PER         94     Australia Perth      16  2016     5477   
446  AUS_PER         94     Australia Perth      16  2016     5558   
447  AUS_PER         94     Australia Perth      16  2016     5560   
448  AUS_PER         94     Australia Perth      16  2016     5278   
449  AUS_PER         94     Australia Perth      16  2016     5559   
450  AUS_ADE         95  Australia Adelaide      16  2016     5215   
451  AUS_ADE         95  Australia Adelaide      16  2016     5001   
452  AUS_ADE         95  Australia Adelaide      16  2016     5019   
453  AUS_ADE         95  Australia Adelaide      16  2016     5479   
454  AUS_ADE         95  Australia Adelaide      16  2016     5477   
455  AUS_ADE         95  Australia Adelaide      16  2016     5213   
456  AUS_ADE         95  Australia Adelaide      16  2016     5708   
457  AUS_ADE         95  Australia Adelaide      16  2016     5709   
458  AUS_ADE         95  Australia Adelaide      16  2016     5478   
459  AUS_ADE         95  Australia Adelaide      16  2016     5555   
460  AUS_ADE        

### Produce Excel Output

In [124]:
#df_pim

In [125]:
#Table positions. 0-based Index. Rows must be >0 as the start marker will be set one row before
df_row=4    
df_col=1
#Table position for a second table at the same worksheet:
df2_row=20
df2_col=1
df3_row=30
df3_col=1
df5_row=50
df5_col=1
# Loop over country codes:
df=pd.read_sql_query(
    """SELECT UPPER(Country_MDD) as CC
    FROM DV_BPM.dbo.Country
    WHERE Country_MDD > ' '
    """, engine_dv_bpm)
for cc in df.CC.unique():
    # Prepare FPI:
    df_fpi_1=df_fpi[((df_fpi.CC==cc) & (df_fpi.Year==p_year)& (df_fpi.LoBID==1))]
    df_fpi_2=df_fpi[((df_fpi.CC==cc) & (df_fpi.Year==p_year)& (df_fpi.LoBID==2))]
    df_fpi_yrs_1=df_fpi_yrs[((df_fpi_yrs.CC==cc) &  (df_fpi_yrs.LoBID==1)& (df_fpi_yrs[p_year].notnull()))]
    df_fpi_yrs_2=df_fpi_yrs[((df_fpi_yrs.CC==cc) &  (df_fpi_yrs.LoBID==2)& (df_fpi_yrs[p_year].notnull()))]
    len_fpi_1=len(df_fpi_1)
    len_fpi_yrs_1=len(df_fpi_yrs_1)
    len_fpi_2=len(df_fpi_2)
    len_fpi_yrs_2=len(df_fpi_yrs_2)
    # Prepare Image Overall:
    df_im_ov_12=df_im_ov[((df_im_ov.CC==cc) & (df_im_ov.Year==p_year))]
    len_im_ov_12=len(df_im_ov_12)
    # Prepare Touchpoint Details:
    df_tp_12=df_tp[((df_tp.CC==cc) & (df_tp.Year==p_year))]
    df_tp_kpi_12=df_tp_kpi[((df_tp_kpi.CC==cc) & (df_tp_kpi.Year==p_year))]
    len_tp_12=len(df_tp_12)
    len_tp_kpi_12=len(df_tp_kpi_12)
    # Prepare PIM vs. Customer share:
    df_pim_12=df_pim[((df_pim.CC==cc) & (df_pim.Year==p_year))]
    df_pim_12_1=df_pim_12[df_pim_12.DeltaPositiv==1]
    df_pim_12_0=df_pim_12[df_pim_12.DeltaPositiv==0]
    len_pim_12=len(df_pim_12)
    len_pim_12_1=len(df_pim_12_1)
    len_pim_12_0=len(df_pim_12_0)
    # print(cc,len_fpi_1,len_fpi_2,len_im_ov_12,len_tp_12,len_pim_12)
    # SetUp Excelfile: (Check if there are data anyway)
    if len_fpi_1+len_fpi_2+len_im_ov_12+len_tp_12+len_pim_12>0:
        writer = pd.ExcelWriter(p_dir+'/'+p_xlsx+'_'+cc+'.xlsx', engine='xlsxwriter')
        workbook  = writer.book
        format = workbook.add_format({'font_color': 'orange','bg_color':'yellow'})
        # FPI: (Check if there are FPI data anyway)
        if len_fpi_1+len_fpi_2>0:
            if len_fpi_1>0:
                #df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==1))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=4, startcol=1)
                #dfp[((dfp.Country==country)& (dfp.LoBID==1)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=20, startcol=1)
                #worksheet0 = writer.sheets['FPI']
                #worksheet0.write_string(2,1, "FPI Life "+country)
                df_fpi_1.sort_values(['FPI'], ascending=[0]).to_excel(writer, sheet_name='FPI_Life',index=False, startrow=df_row, startcol=df_col)
                df_fpi_yrs_1.sort_values([p_year], ascending=[0]).to_excel(writer, sheet_name='FPI_Life',index=False, startrow=df2_row, startcol=df2_col)
                worksheet1 = writer.sheets['FPI_Life']
                worksheet1.write_string(df_row-1,df_col, "FPI_Life_0",format)
                worksheet1.write_string(df_row+len_fpi_1+1,df_col, "FPI_Life_1",format)
                worksheet1.write_string(df2_row-1,df2_col, "FPI_Life_yrs_0",format)
                worksheet1.write_string(df2_row+len_fpi_yrs_1+1,df2_col, "FPI_Life_yrs_1",format)
              #LoBID 2:
            if len_fpi_2>0:
                #df[((df.Country==country) & (df.Year==p_year)& (df.LoBID==2))].sort_values(['Country','LoBID','FPI'], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=34, startcol=1)
                #dfp[((dfp.Country==country)& (dfp.LoBID==2)) ].sort_values(['Country','LoBID',p_year], ascending=[1,1,0]).to_excel(writer, sheet_name='FPI',index=False, startrow=50, startcol=1)
                #worksheet0 = writer.sheets['FPI']
                #worksheet0.write_string(32,1, "FPI P+C "+country)

                df_fpi_2.sort_values(['FPI'], ascending=[0]).to_excel(writer, sheet_name='FPI_PuC',index=False, startrow=df_row, startcol=df_col)
                df_fpi_yrs_2.sort_values([p_year], ascending=[0]).to_excel(writer, sheet_name='FPI_PuC',index=False, startrow=df2_row, startcol=df2_col)
                worksheet2 = writer.sheets['FPI_PuC']
                worksheet2.write_string(df_row-1,df_col, "FPI_PuC_0",format)
                worksheet2.write_string(df_row+len_fpi_2+1,df_col, "FPI_PuC_1",format)
                worksheet2.write_string(df2_row-1,df2_col, "FPI_PuC_yrs_0",format)
                worksheet2.write_string(df2_row+len_fpi_yrs_2+1,df2_col, "FPI_PuC_yrs_1",format)
        # Image Overall:
        if len_im_ov_12>0:
            df_im_ov_12.sort_values(['BrandValue_Non_Customers'], ascending=[0]).to_excel(writer, sheet_name='Image_overall',index=False, startrow=df_row, startcol=df_col)
            worksheet3 = writer.sheets['Image_overall']
            worksheet3.write_string(df_row-1,df_col, "ImageOv_0",format)
            worksheet3.write_string(df_row+len_im_ov_12+1,df_col, "ImageOv_1",format)
        # Touchpoint Details:
        if len_tp_12>0:
            df_tp_kpi_12.sort_values(['KPIsort'], ascending=[1]).to_excel(writer, sheet_name='TP_Details',index=False, startrow=df_row, startcol=df_col)
            df_tp_12.sort_values(['Total'], ascending=[0]).to_excel(writer, sheet_name='TP_Details',index=False, startrow=df2_row, startcol=df2_col)
            worksheet4 = writer.sheets['TP_Details']
            worksheet4.write_string(df_row-1,df_col, "TP_Details_KPIs_0",format)
            worksheet4.write_string(df_row+len_tp_kpi_12+1,df_col, "TP_Details_KPIs_1",format)
            worksheet4.write_string(df2_row-1,df2_col, "TP_Details_0",format)
            worksheet4.write_string(df2_row+len_tp_12+1,df2_col, "TP_Details_1",format)
        # PIM vs. Customer share:
        if len_pim_12>0:
            df_pim_12.sort_values(['PIM'], ascending=[0]).to_excel(writer, sheet_name='PIMvsCS',index=False, startrow=df_row, startcol=df_col)
            df_pim_12_1.to_excel(writer, sheet_name='PIMvsCS',index=False, startrow=df3_row, startcol=df3_col)
            df_pim_12_0.to_excel(writer, sheet_name='PIMvsCS',index=False, startrow=df5_row, startcol=df5_col)
            worksheet5 = writer.sheets['PIMvsCS']
            worksheet5.write_string(df_row-1,df_col, "PIMvsCS_0",format)
            worksheet5.write_string(df_row+len_pim_12+1,df_col, "PIMvsCS_1",format)
            worksheet5.write_string(df3_row-1,df_col, "PIMvsCS_Dpos_0",format)
            worksheet5.write_string(df3_row+len_pim_12_1+1,df_col, "PIMvsCS_Dpos_1",format)
            worksheet5.write_string(df5_row-1,df_col, "PIMvsCS_Dneg_0",format)
            worksheet5.write_string(df5_row+len_pim_12_0+1,df_col, "PIMvsCS_Dneg_1",format)
        writer.save()

In [126]:
import winsound
Freq = 3000 # Set Frequency To 3000 Hertz
Dur = 1000 # Set Duration To 1000 ms == 1 second
#winsound.Beep(Freq,Dur)